## Import Necessary Packages


In [ ]:
import pickle
import numpy as np
import pandas as pd
import chess
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer
from keras.layers import Dropout

<br>

## Import Data Source

In [ ]:
chess_df = pd.read_csv('2014_09_over_1800_one_2100_chess_data.csv')

<br>

## Prepare Data Source For Use In Models

In [ ]:
# Create separate variables for moves, winner, and move number
moves = chess_df['moves']
winner = chess_df['winner']
move_num = chess_df['move_num']
X = []
y = []

In [ ]:
# Check max move number because move numbers need to be removed from moves variable
max(move_num)

162

In [ ]:
# Create list of move numbers to be removed. I made it much higher than the max moves to be safe
char_list = []
for i in range(1,500):
    char_list.append(str(i) + '.')


In [ ]:
# Below code was borrowed from https://towardsdatascience.com/creating-a-chess-engine-with-deep-learning-b9477ff3ee3d

# Matrix formatting function
def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo


# Translate to correct format using chess dict
def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows


# Chess dictionary needed for function    
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

In [ ]:
# Below code was borrowed from https://towardsdatascience.com/creating-a-chess-engine-with-deep-learning-b9477ff3ee3d

# Transform data set to be used in neural net models
for game in moves:
    index = list(moves).index(game)
    allmoves = game.split()
    all_moves = [elem for elem in allmoves if elem not in char_list] 
    total_moves = len(all_moves)
    if winner[index] == 'black':
        game_winner = -1
    else:
        game_winner = 1
    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        value = game_winner * (i/total_moves)
        matrix = make_matrix(board.copy())
        rows = translate(matrix,chess_dict)
        X.append([rows])
        y.append(value)
X = np.array(X).reshape(len(X),8,8,12)
y = np.array(y)
X.shape

(2397813, 8, 8, 12)

<br>

## Convolutional Neural Network Models

In [ ]:
# Baseline CNN model

model = Sequential()

# The input shape is 8,8,12 because it's an 8*8 board with 12 different piece types 
# (king, queen, bishop, knight, rook, pawn) * 2 for each color
model.add(InputLayer(input_shape=(8,8,12)))

model.add(Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(GlobalAveragePooling2D())


model.add(Dense(12, activation='relu'))
model.add(Dense(1,activation = 'tanh'))

model.compile(loss='mse', optimizer='nadam')

model.summary()
model.fit(X, y, epochs=25, verbose=1, validation_split=0.25,
       callbacks=[
           keras.callbacks.ModelCheckpoint(
               'models/chessb.{epoch:02d}-{val_loss:.2f}.hdf5',
               save_best_only=True)
       ])  # track progress as we fit

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 8)           872       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 16)          1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 32)          4640      
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 12)                3

In [ ]:
# Baseline model with three dropout layers added and convolutional filters increased

model = Sequential()

# The input shape is 8,8,12 because it's an 8*8 board with 12 different piece types 
# (king, queen, bishop, knight, rook, pawn) * 2 for each color
model.add(InputLayer(input_shape=(8,8,12)))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(GlobalAveragePooling2D())

model.add(Dropout(0.5))

model.add(Dense(12, activation='relu'))
model.add(Dense(1,activation = 'tanh'))

model.compile(loss='mse', optimizer='nadam')

model.summary()
model.fit(X, y, epochs=25, verbose=1, validation_split=0.25,
       callbacks=[
           keras.callbacks.ModelCheckpoint(
               'models/chessb2.{epoch:02d}-{val_loss:.2f}.hdf5',
               save_best_only=True)
       ])  # track progress as we fit

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 8, 8, 16)          1744      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 16)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 32)          4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 64)         

In [ ]:
# Baseline model with three dropout layers added

model = Sequential()

# The input shape is 8,8,12 because it's an 8*8 board with 12 different piece types 
# (king, queen, bishop, knight, rook, pawn) * 2 for each color
model.add(InputLayer(input_shape=(8,8,12)))

model.add(Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(GlobalAveragePooling2D())

model.add(Dropout(0.5))

model.add(Dense(12, activation='relu'))
model.add(Dense(1,activation = 'tanh'))

model.compile(loss='mse', optimizer='nadam')

model.summary()
model.fit(X, y, epochs=25, verbose=1, validation_split=0.25,
       callbacks=[
           keras.callbacks.ModelCheckpoint(
               'models/chessb3.{epoch:02d}-{val_loss:.2f}.hdf5',
               save_best_only=True)
       ])  # track progress as we fit

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 8, 8, 8)           872       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 4, 16)          1168      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 2, 2, 16)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 32)         

In [ ]:
# Baseline model with one dropout layer added

model = Sequential()

# The input shape is 8,8,12 because it's an 8*8 board with 12 different piece types 
# (king, queen, bishop, knight, rook, pawn) * 2 for each color
model.add(InputLayer(input_shape=(8,8,12)))

model.add(Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(GlobalAveragePooling2D())

model.add(Dropout(0.5))

model.add(Dense(12, activation='relu'))
model.add(Dense(1,activation = 'tanh'))

model.compile(loss='mse', optimizer='nadam')

model.summary()
model.fit(X, y, epochs=25, verbose=1, validation_split=0.25,
       callbacks=[
           keras.callbacks.ModelCheckpoint(
               'models/chessb4.{epoch:02d}-{val_loss:.2f}.hdf5',
               save_best_only=True)
       ])  # track progress as we fit


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 8, 8, 8)           872       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 16)          1168      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 32)          4640      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)               

In [ ]:
# Baseline model with two dropout layers added

model = Sequential()

# The input shape is 8,8,12 because it's an 8*8 board with 12 different piece types 
# (king, queen, bishop, knight, rook, pawn) * 2 for each color
model.add(InputLayer(input_shape=(8,8,12)))

model.add(Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(GlobalAveragePooling2D())


model.add(Dense(12, activation='relu'))
model.add(Dense(1,activation = 'tanh'))

model.compile(loss='mse', optimizer='nadam')

model.summary()
model.fit(X, y, epochs=25, verbose=1, validation_split=0.25,
       callbacks=[
           keras.callbacks.ModelCheckpoint(
               'models/chessb5.{epoch:02d}-{val_loss:.2f}.hdf5',
               save_best_only=True)
       ])  # track progress as we fit

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 8, 8, 8)           872       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 16)          1168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 2, 16)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 32)         